In [36]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import numpy as np
import pandas as pd
import json
import os

In [37]:
# Directory containing the JSON files
dir_path_main = "../results/experiments/"
folders = []
for folder_name in os.listdir(dir_path_main):
    if os.path.isdir(dir_path_main+folder_name):
        folders.append(folder_name)

folder_combined_dfs = []
for folder in folders:
    dir_path = dir_path_main + folder
    # List to store individual DataFrames
    dfs = []
    brute_dfs = []

    for file_name in os.listdir(dir_path):
        if file_name.endswith('.json') and file_name.startswith("brute"):
            file_path = os.path.join(dir_path, file_name)
            with open(file_path) as f:
                data = json.load(f)
                try:
                    size = data["size"][0]
                except:
                    size = np.nan
            data = {key: value for key, value in data.items() if key in {"solver", "time", "best"}}
            brute_df = pd.DataFrame(data)


    # Iterate over each JSON file in the directory
    for file_name in os.listdir(dir_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(dir_path, file_name)
            # Read JSON file into a DataFrame
            with open(file_path) as f:
                data = json.load(f)
            if file_name.startswith("gen"):
                data = {key: value for key, value in data.items() if key in {"solver", "times", "best_final"}}
                df = pd.DataFrame(data)
                
                df["optimum"] = (df['best_final'] == brute_df.loc[0, "best"]).astype(int)
                column_names = {"solvers":"solver", "times":"time", "best_final":"best"}
                df.rename(columns=column_names, inplace=True)
                std = df.groupby("solver").agg({"best":"std"})
                df = df.groupby("solver").agg({"solver" : "min", "time": "mean", "best":"mean", "optimum":"sum"})
                df["optimum"] = df["optimum"] / len(data["best_final"])

                # calculate mean delta to brute solution:
                delta = pd.DataFrame([v - brute_df.loc[0, "best"]  for v in  data["best_final"]]).mean()
                df["delta"] = str(delta[0]) + " ({:.2f})".format(std.iloc[0]["best"])

            else:
                data = {key: value for key, value in data.items() if key in {"solver", "time", "best"}}
                df = pd.DataFrame(data)
                delta = data["best"] - brute_df.loc[0,"best"]
                #std = df.groupby("best").agg(np.std)
                df["delta"] = delta #str(delta[0] + f' ({std})')
                df["optimum"] = (df['best'] == brute_df.loc[0, "best"]).astype(int)
                df["optimum"] = df["optimum"] / len(data["best"])
            
            df["time"] = df["time"].round(2)
            # Append the DataFrame to the list       
            dfs.append(df)      

    # Concatenate all DataFrames in the list into one DataFrame
    combined_df = pd.concat(dfs, ignore_index=True).fillna(0.0)
    combined_df["optimum"] = (combined_df["optimum"]*100).round(0).astype(str) + "%"

    # Set the "solver" column as the index
    combined_df.set_index('solver', inplace=True)

    # Reindex the DataFrame with the specified order
    desired_order = ['brute', 'heuristic', 'plain', 'elitist']
    combined_df = combined_df.reindex(desired_order)
    #combined_df = combined_df.add_suffix("_" + folder)
    combined_df.columns = pd.MultiIndex.from_product([[folder + " size: " + str(size) ], combined_df.columns])

    folder_combined_dfs.append(combined_df)


 
# Concatenate the DataFrames
combined_df = pd.concat(folder_combined_dfs, axis=1)
combined_df.rename(columns={"time":"time in sec", "optimum" : "optimum perc."}, inplace=True)


# Display the combined DataFrame
combined_df.T


solver                                           brute heuristic  \
fully_synthetic size: 7779240  time in sec    62618.45      0.01   
                               best              357.0     370.0   
                               delta               0.0      13.0   
                               optimum perc.    100.0%      0.0%   
no_deletes size: 691200        time in sec     1301.12      0.01   
                               best              492.0     492.0   
                               delta               0.0       0.0   
                               optimum perc.    100.0%    100.0%   
heterogen size: 537600         time in sec      1661.0      0.01   
                               best              491.0     492.0   
                               delta               0.0       1.0   
                               optimum perc.    100.0%      0.0%   
close_maxima size: 4147200     time in sec    13904.79      0.01   
                               best              172.0     183.0   
                               delta               0.0      11.0   
                               optimum perc.    100.0%      0.0%   
invalid_branches size: 1658880 time in sec     6198.89      0.01   
                               best              187.0     253.0   
                               delta               0.0      66.0   
                               optimum perc.    100.0%      0.0%   

solver                                               plain       elitist  
fully_synthetic size: 7779240  time in sec           33.88         10.04  
                               best                  410.2         360.8  
                               delta          53.2 (32.31)    3.8 (5.25)  
                               optimum perc.          0.0%         30.0%  
no_deletes size: 691200        time in sec           11.43           2.7  
                               best                  568.3         506.3  
                               delta          76.3 (31.33)  14.3 (19.49)  
                               optimum perc.          0.0%         50.0%  
heterogen size: 537600         time in sec           18.86          4.34  
                               best                  536.3         498.5  
                               delta          45.3 (32.62)    7.5 (9.69)  
                               optimum perc.          0.0%         60.0%  
close_maxima size: 4147200     time in sec            6.88           3.8  
                               best                  172.0         172.0  
                               delta            0.0 (0.00)    0.0 (0.00)  
                               optimum perc.        100.0%        100.0%  
invalid_branches size: 1658880 time in sec           22.81          4.83  
                               best                  187.0         187.0  
                               delta            0.0 (0.00)    0.0 (0.00)  
                               optimum perc.        100.0%        100.0%